# Imports and helper functions

In [328]:
import sqlalchemy
import sqlite3
from sklearn.cluster import KMeans
import umap
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from datetime import datetime as dt
from sklearn import metrics
import seaborn as sns
import snakecase
import re
import numpy as np
import dtype_diet
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import joblib
import s3fs
from dotenv import load_dotenv
import os

# Loading data

In [2]:
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
s3 = s3fs.S3FileSystem(
    anon=False, key=AWS_ACCESS_KEY_ID, secret=AWS_SECRET_ACCESS_KEY)

In [78]:
load_dotenv()
 
HOST = os.getenv('HOST')
USER = os.getenv('USER')
PASSWORD = os.getenv('PASSWORD')

endpoint = 'postgresql://{}:{}@{}:5432/postgres'.format(USER, PASSWORD, HOST)

conn = sqlalchemy.create_engine(endpoint).connect()

query = """
SELECT * from customers
"""

df = pd.read_sql_query(sqlalchemy.text(query), con=conn)

# Data Wrangling

## invoice_no

In [79]:
df = df[~df['invoice_no']
    .astype(str)
    .apply(lambda x: bool(re.search('[^0-9]+', x)))
]

## stock_code

In [80]:
df = df[~df['stock_code'].isin(
    ['DOT', 'BANK CHARGES', 'AMAZONFEE', 'PADS', 'POST', 'M', 'D', 'm']
    )
]

## invoice_date

In [81]:
df['invoice_date'] = df['invoice_date'].apply(
    lambda x: dt.strptime(x, '%d-%b-%y')
)

In [82]:
df = df[df['quantity'] > 0]
df = df[df['unit_price'] != 0]

## Checking/Replace NAs

In [83]:
df_aux = pd.DataFrame(
    df[df['customer_id'].isna()]['invoice_no']
    .drop_duplicates()
    )
df_aux = df_aux.assign(customer_id=np.arange(20000, 20000 + len(df_aux), 1))   


df = pd.merge(df,df_aux, how='left', on='invoice_no')
df['customer_id'] = (df['customer_id_x']
                            .combine_first(df['customer_id_y'])
                            .astype(int)
)
df = df.drop(columns=['customer_id_x', 'customer_id_y'], axis=1)

# Feature Engineering

In [84]:
df1 = df.drop(columns=['description', 'country'])
del(df)

## Gross Revenue

In [86]:
df_purchases = df1[['customer_id','invoice_no']].drop_duplicates()
df1_aux = (df1.loc[:,['invoice_no','quantity','unit_price']]
            .assign(gross_revenue=df1['quantity']*df1['unit_price'])
)                   
df1_aux = (df1_aux[['invoice_no','gross_revenue']]
            .groupby('invoice_no')
            .sum()
            .reset_index()
)
df_purchases = pd.merge(df_purchases,df1_aux, how='left', on='invoice_no')
df1_1 = (df_purchases[['customer_id','gross_revenue']]
        .groupby('customer_id')
        .sum()
        .reset_index()
)

##  Recency

In [89]:
df1_aux = (df1[['customer_id', 'invoice_date']]
           .groupby('customer_id')
           .max()
           .reset_index()
)
df1_aux['recency_days'] = (df1['invoice_date'].max()
                                                - df1_aux['invoice_date']).dt.days 

df1_1 = pd.merge(
    df1_1, df1_aux[['customer_id', 'recency_days']],
    on='customer_id', how='left')

## Quantity of purchases

In [91]:
df1_aux = (df1[['customer_id','invoice_no']]
            .groupby('customer_id')
            .nunique()
            .reset_index()
            .rename(columns={'invoice_no': 'qtd_purchases'})
)
df1_1 = pd.merge(df1_1, df1_aux, on='customer_id', how='left')

## Quantity of products

In [92]:
df1_aux = (df1[['customer_id', 'stock_code']]
            .groupby('customer_id')
            .nunique()
            .reset_index()
            .rename(columns={'stock_code':'qtd_products'})
)
df1_1 = pd.merge(df1_1, df1_aux, on='customer_id', how='left')

# Data Preparation

In [240]:
df2 = df1_1.copy()
df2 = df2.drop(columns='customer_id')
del(df1)

In [241]:
# rs = RobustScaler()
# df2['gross_revenue'] = rs.fit_transform(
#     df2[['gross_revenue']].values)
# joblib.dump(rs,s3.open(
#     's3://insiders-customers-dataset/gross_revenue_scaler.pkl', 'wb')
# )

# df2['recency_days'] = rs.fit_transform(
#     df2[['recency_days']].values)
# joblib.dump(rs,s3.open(
#     's3://insiders-customers-dataset/recency_days_scaler.pkl', 'wb')
# )


# df2['qtd_purchases'] = rs.fit_transform(
#     df2[['qtd_purchases']].values)
# joblib.dump(rs,s3.open(
#     's3://insiders-customers-dataset/qtd_purchases_scaler.pkl', 'wb')
# )


# df2['qtd_products'] = rs.fit_transform(
#     df2[['qtd_products']].values)
# joblib.dump(rs,s3.open(
#     's3://insiders-customers-dataset/qtd_products_scaler.pkl', 'wb')
# )


df2['gross_revenue'] = (joblib.load(
    s3.open('s3://insiders-customers-dataset/gross_revenue_scaler.pkl', 'rb')
     )
    .transform(df2[['gross_revenue']].values)
)


df2['recency_days'] =  (joblib.load(
    s3.open('s3://insiders-customers-dataset/recency_days_scaler.pkl', 'rb')
     )
    .transform(df2[['recency_days']].values)
)


df2['qtd_purchases'] =  (joblib.load(
    s3.open('s3://insiders-customers-dataset/qtd_purchases_scaler.pkl', 'rb')
     )
    .transform(df2[['qtd_purchases']].values)
)


df2['qtd_products'] =  (joblib.load(
    s3.open('s3://insiders-customers-dataset/qtd_products_scaler.pkl', 'rb')
     )
    .transform(df2[['qtd_products']].values)
)

In [321]:
# pipeline = Pipeline(
#     steps = [
#         ('preprocessor', RobustScaler()),
#         ('umap_reducer', umap.UMAP(random_state=42))
#     ]
# )
# embedding_umap = pipeline.fit(df2)
# joblib.dump(
#     embedding_umap,s3.open('s3://insiders-customers-dataset/embedding_umap.pkl', 'wb')
# )

embedding_umap = (joblib.load(
    s3.open('s3://insiders-customers-dataset/embedding_umap.pkl', 'rb')
    )
    .transform(df2)
)

df_umap = pd.DataFrame()
df_umap['embedding_x'] = embedding_umap[:, 0]
df_umap['embedding_y'] = embedding_umap[:, 1]

Exception during reset or similar
Traceback (most recent call last):
  File "/home/dbcordeiro/anaconda3/envs/high_value_customer_identification/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 988, in _finalize_fairy
    fairy._reset(
  File "/home/dbcordeiro/anaconda3/envs/high_value_customer_identification/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 1437, in _reset
    pool._dialect.do_rollback(self)
  File "/home/dbcordeiro/anaconda3/envs/high_value_customer_identification/lib/python3.10/site-packages/sqlalchemy/engine/default.py", line 693, in do_rollback
    dbapi_connection.rollback()
psycopg2.OperationalError: SSL SYSCALL error: EOF detected



# Model Training

## Final model

In [322]:
df4 = df_umap

In [324]:
# kmeans_model = KMeans(n_clusters=5, random_state=42, init='k-means++').fit(df4)
# joblib.dump(kmeans_model,
#     s3.open('s3://insiders-customers-dataset/kmeans_model.pkl', 'wb')
#     )

labels  = joblib.load(
    s3.open('s3://insiders-customers-dataset/kmeans_model.pkl', 'rb')
    ).predict(df4)

df4['cluster'] = labels

# Cluster analysis

## Cluster profile

In [325]:
sns.scatterplot(x= 'embedding_x',y= 'embedding_y', hue='cluster',data= df4, palette='deep');

ValueError: Could not interpret value `cluster` for parameter `hue`

In [307]:
final_metric = metrics.silhouette_score(df4, labels=labels, metric='euclidean')
final_metric

0.5251477907428741

In [326]:
#removing fake customers
df5 = df1_1.copy()
df5['cluster'] = labels
df5 = df5[df5['customer_id'] < 20000]
df5['cluster_date'] = dt.strftime(dt.now(), '%y-%b-%d')

In [327]:
df_cluster = (df5[['customer_id', 'cluster']]
                .groupby('cluster')
                .count()
                .reset_index()
)
df_cluster = df_cluster.assign(
    perc_customer=100*(df_cluster['customer_id'] / 
                                            df_cluster['customer_id'].sum())
)
df_avg_gross_revenue = (df5[['cluster', 'gross_revenue']]
                                    .groupby('cluster')
                                    .median()
                                    .reset_index()
)
df_cluster = pd.merge(
    df_cluster, df_avg_gross_revenue, how='inner', on='cluster')

# Avg recency
df_recency = df5[['cluster', 'recency_days']].groupby(
    'cluster').median().reset_index()
df_cluster = pd.merge(
    df_cluster, df_recency, how='inner', on='cluster')

# Avg quantity of purchases
df_recency = (df5[['cluster', 'qtd_purchases']]
                .groupby('cluster')
                .median()
                .reset_index()
)
df_cluster = pd.merge(
    df_cluster, df_recency, how='inner', on='cluster')

# Avg quantity of products
df_qtd_products = (df5[['cluster', 'qtd_products']]
                            .groupby('cluster')
                            .median()
                            .reset_index()
)
df_cluster = pd.merge(
    df_cluster, df_qtd_products, how='inner', on='cluster')

df_cluster_result = df_cluster.sort_values(
    by='gross_revenue', ascending=False)
display(df_cluster_result)

,cluster,customer_id,perc_customer,gross_revenue,recency_days,qtd_purchases,qtd_products
0,0,1073,24.76,2984.35,15.00,8.00,119.00
4,4,957,22.08,981.65,38.00,4.00,47.00
2,2,839,19.36,547.07,64.00,2.00,30.00
1,1,883,20.37,262.20,61.00,1.00,17.00
3,3,582,13.43,232.97,281.00,1.00,14.00


# Deploy to production

## Insert into SQLITE

In [292]:
query_drop_table = """
CREATE TABLE insiders (
    customer_id INT,
    gross_revenue REAL,
    recency_days INT,
    qtd_purcases INT,
    qtd_products INT,
    cluster INT,
    cluster_date VARCHAR(20)
)
"""

In [293]:
load_dotenv()
 
host = os.getenv('HOST')
user = os.getenv('USER')
password = os.getenv('PASSWORD')

endpoint = 'postgresql://{}:{}@{}:5432/postgres'.format(user, password, host)

conn = sqlalchemy.create_engine(endpoint).connect()
conn.execute(sqlalchemy.text(query_drop_table))
conn.commit()
conn.close()

In [294]:
load_dotenv()
 
host = os.getenv('HOST')
user = os.getenv('USER')
password = os.getenv('PASSWORD')

endpoint = 'postgresql://{}:{}@{}:5432/postgres'.format(user, password, host)

conn = sqlalchemy.create_engine(endpoint)
# conn.connect()
# conn.execute(sqlalchemy.text(query_create_table))
# conn.commit()
# conn.close()


df5.to_sql(
    'insiders', con=conn, if_exists='replace', index=False)

334